<a href="https://colab.research.google.com/github/vrao9/google_colab/blob/master/mnist_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google colab TPU for MNIST dataset
This notebook can be run on google colab's TPU. In edit, notebook setting, choose TPU
Imports and choose TPU

In [0]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

Data is scaled so that the pixel values lie between 0 and 1

In [0]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

**Create a model for given batch size**. \
Comment: Here (784,) is used because the function does not accept integers, but only tupple!! 
For the input shape, the batch size should not be considered!!

In [0]:
def create_model(batch_size = None):
  source = tf.keras.layers.Input(shape = (28,28), batch_size = batch_size, name = "Input" )
  x_reshape = tf.keras.layers.Reshape((784,), input_shape=(28,28))(source)
  x_dense_1 = tf.layers.Dense(512, activation = tf.nn.relu, input_dim = 784)(x_reshape)
  x_dense_2 = tf.layers.Dense(10, activation=tf.nn.softmax)(x_dense_1)
  model = tf.keras.Model(inputs=[source], outputs=[x_dense_2])
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

Clear the previous session and use the previously defined function to create the **network model.**

In [4]:
tf.keras.backend.clear_session()
model = create_model(batch_size = 128)
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (128, 28, 28)             0         
_________________________________________________________________
reshape (Reshape)            (128, 784)                0         
_________________________________________________________________
dense_1 (Dense)              (128, 512)                401920    
_________________________________________________________________
dense_2 (Dense)              (128, 10)                 5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Get the** TPU address** and the** number of cores** that are available! \\
Usually there are 8 cores that are available in the TPU. The batch size **should be** preferably multiples of 64 for small dataset (vectorization speeds up the process)

In [5]:
import os
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.summary()

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.77.218.122:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 5485425033211786578)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 7026917199547085571)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 12776172722290479229)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9281614165039328125)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5195973696579127192)
INFO:tensorflow:*** Available Device: _Device

Train the model and save the weights at the end of each epoch.

In [6]:
import time
start_time = time.time()

history = tpu_model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128 * 8)
tpu_model.save_weights('./tpu_model.h5', overwrite=True)

print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(128,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(128, 28, 28), dtype=tf.float32, name='Input_10'), TensorSpec(shape=(128, 1), dtype=tf.float32, name='dense_2_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Cloning Adam {'lr': 0.0010000000474974513, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'decay': 0.0, 'epsilon': 1e-07, 'amsgrad': False}
INFO:tensorflow:Remapping placeholder for Input
INFO:tensorflow:KerasCrossShard: <tensorflow.python.keras.optimizers.Adam object at 0x7fe60c4bacc0> []
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 1.1876811981201172 secs
INFO:tensorflow:Setting weights on TPU model.
INFO:tensorflow:CPU -> TPU lr: 0.0010000000474974513 {0.001}
INFO:tensorflow:CPU -> TPU beta_1: 0.8999999761581421 {0.9}
INFO:tensorflow:CPU -> TPU beta_2: 0.9990000128746033 {0.999}
INFO:te

For inference, it's best option to create a new model and load the weights which has been previously obtained and use these to get the inference of the model.

In [7]:
inferencing_model = create_model()
inferencing_model.load_weights('./tpu_model.h5')
inferencing_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 28, 28)            0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [8]:
inferencing_model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 97us/step


[0.059936321102594954, 0.9809]